In [5]:
from datetime import time
!pip install pyspark==3.5.6 --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.4/317.4 MB 33.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.6-py2.py3-none-any.whl size=317895856 sha256=86c472be7bbdd71c72337c420b1483e6fd57adf713b0405331664fbdd2c9e003
  Stored in directory: /root/.cache/pip/wheels/64/62/f3/ec15656ea4ada0523cae62a1827fe7beb55d3c8c87174aad4a
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.9
    Uninstalling py4j-0.10.9.9:
      Successfully uninstalled py4j-0.10.9.9
  Attempting uninstall: pyspark
    Found existing installation: pyspark 4.0.0
    Uninstalling pyspark-4.0.0:
      Successfully uninstalled pyspark-4.0.0

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# %load_ext autoreload
# %autoreload 2
# %reload_ext autoreload

import os
ROOT_DIR = '/workspace/NN'
os.chdir(ROOT_DIR)

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


---------------

In [2]:

import os
ROOT_DIR = '/workspace/NN'
os.chdir(ROOT_DIR)
from pyspark.sql import SparkSession
# driver_host = "localhost"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["KUBECONFIG"] = os.path.join(ROOT_DIR, "k8s-creds", 'config')
os.environ["PYSPARK_SUBMIT_ARGS"] = f"""
--conf spark.executor.memory=1G
--conf spark.executor.cores=1
pyspark-shell
"""
spark = (SparkSession.builder
    .appName("k8s-test-app")
    # .master("k8s://https://kubernetes.docker.internal:6443")
.master("spark://host.docker.internal:30077")
    .config("spark.kubernetes.container.image", "bitnami/spark:latest")
    .config("spark.kubernetes.namespace", "default")
    .config("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")

        .config("spark.executor.instances", "2")      # сколько Executors запустить
        .config("spark.executor.cores", "1")          # по одному CPU‐ядру
        .config("spark.executor.memory", "1g")      # по 512 МБ памяти
        .config("spark.driver.memory", "1g")        # драйверу тоже ограничим RAM, если нужно

    # .config("spark.kubernetes.authenticate.submission.caCertFile",  os.path.join(ROOT_DIR, 'k8s-creds', "ca.crt"))
    # .config("spark.kubernetes.authenticate.submission.clientKeyFile",  os.path.join(ROOT_DIR, 'k8s-creds', "client.key"))
    # .config("spark.kubernetes.authenticate.submission.clientCertFile",  os.path.join(ROOT_DIR, 'k8s-creds', "client.crt"))
    # .config("spark.kubernetes.executor.podNamePrefix", "spark-exec")

    .config("spark.driver.host", "host.docker.internal")
    .config("spark.jars.ivy", "/tmp/.ivy2")  # укажи директорию вручную

    # .config("spark.submit.deployMode", "cluster")

    .config("spark.driver.bindAddress",  "0.0.0.0")
    .config("spark.driver.port",         "45555")
    .config("spark.blockManager.port",   "45556")

    # фиксированные порты, если нужны

    # .config("spark.driver.bindAddress", "0.0.0.0")
    # .config("spark.eventLog.enabled", "true")
    # .config("spark.ui.showConsoleProgress", "true")
    # .config("spark.driver.extraJavaOptions", "-Dlog4j.configuration=file:/path/to/log4j.properties")
    .getOrCreate())


# spark = (SparkSession.builder
#              .appName("k8s-test-app")
#     .master("k8s://https://kubernetes.docker.internal:6443")
#
#     # образ и namespace
#     .config("spark.kubernetes.container.image", "bitnami/spark:latest")
#     .config("spark.kubernetes.namespace", "spark")
#     .config("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")
# # .config("spark.kubernetes.client.watch.allowWatchBookmarks", "false")
#     .config("spark.submit.deployMode", "client")
#     # .config("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")
#
#
#     # ресурсы
#     .config("spark.executor.instances", "2")
#
#     # драйвер: рекламируемый адрес (для исполнителей) и bind address (куда слушать)
#     # .config("spark.driver.host", "host.docker.internal")
#     # .config("spark.driver.bindAddress", "0.0.0.0")
#     #
#     # # фиксированные порты, если нужны
#     # .config("spark.driver.port", "45555")
#     # .config("spark.blockManager.port", "45556")
#
#     .getOrCreate())

df = spark.range(1000)
df.selectExpr("sum(id)").show()

spark.stop()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/07 20:44:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-------+
|sum(id)|
+-------+
| 499500|
+-------+

